In [9]:
import numpy as np
from scipy import optimize
import pykep

In [73]:
def get_dW(r1,r2):
    print("alpha",np.arccos(np.dot(r1,r2)))
    return np.arccos(np.dot(r1,r2))

In [34]:
def get_r(r1,r2, t, mu =398600):
    return (t*np.sqrt(mu)/get_dW(r1,r2))**(2/3)


In [67]:
def get_rho(r_obs,r_tar_hat, r):
    c = np.dot(r_obs,r_tar_hat)
    return -c + np.sqrt(c**2 - np.linalg.norm(r_obs)**2 + r**2)

In [13]:
def get_target_r(r_obs,r_tar_hat, r):
    rho = get_rho(r_obs,r_tar_hat, r)
    return r_obs + rho*r_tar_hat

In [14]:
def propagate_based_on_elements(elements, t, kepler=True):
    r, v = pykep.par2ic(elements, mu=398600)
    if t > 0:
        rf,vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    else:
        rf, vf = r, v
    if kepler == False:  # return vectors
        return np.array(rf),np.array(vf)
    if kepler == True:  # return kepler elements
        return pykep.ic2par(rf,vf,mu = 398600)

In [15]:
def get_v(r1,r2,r, mu=398600):
    h = np.cross(r1,r2)/np.linalg.norm(np.cross(r1,r2))
    v_hat = np.cross(h,r2)/r
    return v_hat * np.sqrt(mu/r)

In [65]:
def circular_method(measurements, times, elements_obs):
    sol =[]
    all_r = []
    for i in range(len(measurements)):
        L1_hat = measurements[i]
        t1 = times[i]
        for j in range(len(measurements)):
            if j != i:
                tau = times[j] - t1
                if tau > 0:
                    r = get_r(L1_hat, measurements[j],tau)
                    print("r", r)
                    all_r.append(r)

    r = sum(all_r)/len(all_r)
    print("average r",r)
    for i in range(len(measurements)):
        r_obs, v_obs = propagate_based_on_elements(elements_obs,times[i],kepler=False)
        r_tar = get_target_r(r_obs, measurements[i],r)
        v_tar = get_v(r_obs,r_tar,r)
        sol.append([r_tar,v_tar])

    return sol

In [77]:
elements_sat = [7350,0.0037296,np.radians(82.0394),np.radians(41.7294),np.radians(216.168),np.radians(0)]
elements_deb = [7500,0,np.radians(2),np.radians(10),np.radians(0),np.radians(45)]
times = [0,400,1200]
measurements = []
for t in times:
    r_sat, v_sat = propagate_based_on_elements(elements_sat,t,kepler=False)
    r_deb, v_deb = propagate_based_on_elements(elements_deb,t,kepler=False)
    print("debris r", r_deb)
    print("r", np.linalg.norm(r_deb))
    print("debrs v",v_deb)
    L = r_deb - r_sat
    
    L_hat = L/np.linalg.norm(L)
    measurements.append(L_hat)

debris r [4302.38426523 6140.45878507  185.08253084]
r 7500.0
debrs v [-5.97121731  4.17838065  0.17990456]
debris r [1652.49772647 7311.70038772  241.4305371 ]
r 7500.000000000001
debrs v [-7.11088057  1.60386589  0.09827721]
debris r [-3954.9259052   6367.8508586    242.97433407]
r 7500.000000000001
debrs v [-6.19404713 -3.84337026 -0.09461419]


In [78]:
sol = circular_method(measurements,times,elements_sat)
sol

alpha 0.35195838591678985
r 8014.778923289636
alpha 1.2934687354682435
r 7000.503949263067
alpha 0.9415350661114256
r 6602.089739688799
average r 7205.790870747168


[[array([4120.7301658 , 5910.61251214,   87.54830054]),
  array([ 1.07010353, -0.85433092,  7.3103774 ])],
 [array([1550.81326143, 7036.5741081 ,   70.88809733]),
  array([ 1.67351719, -0.44170197,  7.23331599])],
 [array([-3737.88979696,  6160.46539601,   -16.37203015]),
  array([-1.83492674, -1.09441715,  7.12403787])]]

https://apps.dtic.mil/sti/pdfs/AD1121954.pdf  -  
https://www.sciencedirect.com/science/article/pii/S0273117721005287#e0015  - another novel one
https://www.sciencedirect.com/science/article/pii/B9780123747785000052#b0160  - the book
https://www.researchgate.net/publication/254199485_Modifications_to_the_Gooding_Algorithm_for_Angles-Only_Initial_OrbitDetermination  - Cooding
file:///C:/Users/Admin/Downloads/Thesis_Bert_Van_den_Abbeele_v2.pdf  - Delft
https://conference.sdo.esoc.esa.int/proceedings/sdc8/paper/129/SDC8-paper129.pdf  - cooding useless?
